Importar librerias

In [1]:
import tensorflow
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

Cargar datos

In [3]:
data = pd.read_csv('drive/MyDrive/HAM10000/HAM10000_metadata.csv')
data['image_full_name'] = data['image_id']+'.jpg'

Creación del modelo

In [4]:
modelo = Sequential()
modelo.add(Conv2D(64, (3, 3), activation='relu'))
modelo.add(MaxPooling2D(pool_size=(2, 2)))
modelo.add(Dropout(0.5))
modelo.add(Flatten())
modelo.add(Dense(128, activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(7, activation='sigmoid'))

Dividir datos: entrenamiento, prueba y validación

In [5]:
X = data[['image_full_name','dx','lesion_id']]
Y = X.pop('dx').to_frame()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val   = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

In [6]:
train = pd.concat([X_train,y_train],axis=1)
val = pd.concat([X_val,y_val],axis=1)
test = pd.concat([X_test,y_test],axis=1)

In [7]:
encoder = LabelEncoder()
encoder.fit(val['dx'])
name_as_indexes_train = encoder.transform(val['dx']) 
val['label'] = name_as_indexes_train

In [8]:
encoder = LabelEncoder()
encoder.fit(test['dx'])
name_as_indexes_test = encoder.transform(test['dx']) 
test['label'] = name_as_indexes_test

In [9]:
train_generator = ImageDataGenerator(rescale=1./255, rotation_range=10, zoom_range = 0.1, 
                                     width_shift_range=0.0, height_shift_range=0.00)

In [10]:
train_data = train_generator.flow_from_dataframe(dataframe=train, x_col="image_full_name", y_col="dx",
                                                batch_size=32, directory="drive/MyDrive/HAM10000/HAM10000_images",
                                                shuffle=True, class_mode="categorical", target_size=(224,224))

Found 7235 validated image filenames belonging to 7 classes.


In [12]:
test_generator = ImageDataGenerator(rescale=1./255,rotation_range=10, zoom_range=0.1, 
                                    width_shift_range=0.0, height_shift_range=0.02)

In [13]:
test_data = test_generator.flow_from_dataframe(dataframe=test, x_col="image_full_name", y_col="dx",
                                              directory="drive/MyDrive/HAM10000/HAM10000_images",
                                              shuffle=False, batch_size=1, class_mode=None, target_size=(224,224))

Found 1503 validated image filenames.


In [14]:
val_data = test_generator.flow_from_dataframe(dataframe=val, x_col="image_full_name", y_col="dx",
                                              directory="drive/MyDrive/HAM10000/HAM10000_images",
                                              batch_size=64, shuffle=False, class_mode="categorical", target_size=(224,224))

Found 1277 validated image filenames belonging to 7 classes.


Compilar modelo

In [15]:
sgd = optimizers.SGD(learning_rate=0.01, clipvalue=0.5)
modelo.compile(optimizer=sgd,loss="categorical_crossentropy",metrics=["accuracy"])

Entrenar modelo

In [17]:
history = modelo.fit_generator(generator=train_data, steps_per_epoch=train_data.samples//train_data.batch_size,
                              validation_data=val_data, verbose=1, validation_steps=val_data.samples//val_data.batch_size,
                              epochs=3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/3
226/226 [==============================] - 3154s 14s/step - loss: 1.1450 - accuracy: 0.6661 - val_loss: 1.0968 - val_accuracy: 0.6752
Epoch 2/3
226/226 [==============================] - 770s 3s/step - loss: 1.0194 - accuracy: 0.6706 - val_loss: 0.9324 - val_accuracy: 0.6752
Epoch 3/3
226/226 [==============================] - 774s 3s/step - loss: 0.9802 - accuracy: 0.6703 - val_loss: 0.9026 - val_accuracy: 0.6752


Evaluación porcentaje de precisión

In [18]:
test_data.reset()
predictions = modelo.predict_generator(test_data, steps=test_data.samples/test_data.batch_size, verbose=1)
y_pred = np.argmax(predictions, axis=1)
c=np.where(y_pred == name_as_indexes_test)
print("Test Accuracy :",(np.count_nonzero(c)/len(name_as_indexes_test))*100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


1503/1503 [==============================] - 518s 345ms/step
Test Accuracy : 66.40053226879574


Probar con una imagen en especifico

In [23]:
def predecir(direccion_img):
  lesion_classes_dict = {
    0 : 'Melanocytic nevi',
    1 : 'Melanoma',
    2 : 'Benign keratosis-like lesions ',
    3 : 'Basal cell carcinoma',
    4 : 'Actinic keratoses',
    5 : 'Vascular lesions',
    6 : 'Dermatofibroma'
  }

  img = tensorflow.keras.preprocessing.image.load_img(direccion_img, target_size=(224,224,3))

  x = tensorflow.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  preds = modelo.predict(x)
  pred_class = preds.argmax(axis=-1)      

  pr = lesion_classes_dict[pred_class[0]]
  resultado =str(pr)  
  return resultado

In [33]:
print(predecir("drive/MyDrive/HAM10000/HAM10000_images/ISIC_0024318.jpg"))

1/1 [==============================] - 0s 83ms/step
Melanoma


In [34]:
print(predecir("drive/MyDrive/HAM10000/HAM10000_images/ISIC_0025967.jpg"))

1/1 [==============================] - 0s 87ms/step
Melanoma


In [35]:
print(predecir("drive/MyDrive/HAM10000/HAM10000_images/ISIC_0026627.jpg"))

1/1 [==============================] - 0s 148ms/step
Benign keratosis-like lesions 


In [36]:
print(predecir("drive/MyDrive/HAM10000/HAM10000_images/ISIC_0031144.jpg"))

1/1 [==============================] - 0s 123ms/step
Melanocytic nevi


In [37]:
print(predecir("drive/MyDrive/HAM10000/HAM10000_images/ISIC_0031364.jpg"))

1/1 [==============================] - 0s 115ms/step
Benign keratosis-like lesions 


Guardar modelo

In [21]:
modelo.save('Trabajo_Final.h5')